In [1]:
!sudo echo -ne '\n' | sudo add-apt-repository ppa:alessandro-strada/ppa >/dev/null 2>&1 # note: >/dev/null 2>&1 is used to supress printing
!sudo apt update >/dev/null 2>&1
!sudo apt install google-drive-ocamlfuse >/dev/null 2>&1
!google-drive-ocamlfuse
!sudo apt-get install w3m >/dev/null 2>&1 # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop >/dev/null 2>&1 # to set default browser 
%cd /content
!mkdir gdrive
%cd gdrive
!mkdir "My Drive"
!google-drive-ocamlfuse "/content/gdrive/My Drive"

/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force&state=qu9XFcz8T43uzUHKdzImYH8XDELfjnMpIWuQ6EeBL6I'
/bin/sh: 1: firefox: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: chromium-browser: not found
/bin/sh: 1: open: not found
Cannot retrieve auth tokens.
Failure("Error opening URL:https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.go

Setting root path for the project

In [2]:
root_path = "/content/gdrive/My Drive/Project_Dataset_MSVD"
!pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=bd74203a2f34a7ceddfecbefe5b8e30e6625c53314e0e52130d38f12075fc83d
  Stored in directory: /root/.cache/pip/wheels/30/33/46/5ab7eca55b9490dddbf3441c68a29535996270ef1ce8b9b6d7
Successfully built ffmpeg


importing libraries

In [3]:
import subprocess, ffmpeg, multiprocessing, os, shutil,cv2, json
from multiprocessing import Pool
import numpy as np
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
import torch as th
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.engine.training import Model
from keras.layers.regularization.spatial_dropout3d import Dropout
from keras.layers.core.dense import Dense
from keras.layers.reshaping.flatten import Flatten
from keras.layers.pooling.average_pooling2d import AveragePooling2D
from keras.layers import Input
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from numpy import dot
from numpy.linalg import norm

Function to compress videos

In [ ]:
def compress_video(input_path, output_path):
  try:
    command = [
        'ffmpeg',
        '-y','-i', input_path,
        '-filter:v',
        'scale=\'if(gt(a,1),trunc(oh*a/2)*2,224)\':\'if(gt(a,1),224,trunc(ow*a/2)*2)\'',
        '-map','0:v',
        '-r', '5',
        output_path,
    ]
    ffmpeg = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = ffmpeg.communicate()
    return ffmpeg.poll()
  except Exception as e:
    raise e

Defining directories for training and test sets for videos

In [4]:
train_video_path = root_path + "/train_videos/train_videos"
test_video_path = root_path + "/test_videos/test_videos"
cmpd_train_video_path = root_path + "/ten_class_train_set"
cmpd_test_video_path = root_path + "/ten_class_test_set"
input_path_list = []

getting annotation data on train set

In [5]:
train_anno = root_path + "/train_annotations/train_annotations.json"
json_file = open(train_anno)
annotations = json.load(json_file)
json_file.close()

creating annotation dictionary to be used later

In [6]:
anno_dict = {}
for item in annotations["videos"]:
  anno_dict[item['video_id'] + '.mp4'] = item["category"]

filtering out 50 videos of first 5 categories

In [ ]:
train_set = [
    [item for item in annotations["videos"] if item["category"] == 0][0:10],
    [item for item in annotations["videos"] if item["category"] == 8][0:10],
    [item for item in annotations["videos"] if item["category"] == 9][0:10],
    [item for item in annotations["videos"] if item["category"] == 11][0:10],
    [item for item in annotations["videos"] if item["category"] == 16][0:10],
    ]

In [ ]:
train_set = sum(train_set, [])
len(train_set)

50

In [ ]:
for row in train_set:
  name = row["video_id"]
  input_file_name = train_video_path + "/" + name + ".mp4"
  output_file_name = cmpd_train_video_path + "/" + name + ".mp4"
  print(output_file_name)
  compress_video(input_file_name, output_file_name)

/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video27.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video43.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video47.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video48.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video56.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video66.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video97.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video106.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video119.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video127.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video3.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_set/video71.mp4
/content/gdrive/My Drive/Project_Dataset_MSVD/ten_class_train_

Filtering out test set

In [ ]:
test_set = [
    [item for item in annotations["videos"] if item["category"] == 0][10:21],
    [item for item in annotations["videos"] if item["category"] == 8][10:21],
    [item for item in annotations["videos"] if item["category"] == 9][10:21],
    [item for item in annotations["videos"] if item["category"] == 11][10:21],
    [item for item in annotations["videos"] if item["category"] == 16][10:21],
    ]

In [ ]:
test_set = sum(test_set, [])
len(test_set)

55

In [ ]:
for row in test_set:
  name = row["video_id"]
  input_file_name = train_video_path + "/" + name + ".mp4"
  output_file_name = cmpd_test_video_path + "/" + name + ".mp4"
  compress_video(input_file_name, output_file_name)

code snippet to convert video to tensor :-

transform_image function is used to apply transformations and normalizations to the image


In [ ]:
def transform_image(resolution):
  result = Compose([
      Resize(resolution, interpolation=Image.BICUBIC),
      CenterCrop(resolution),
      lambda image: image.convert("RGB"),
      ToTensor(),
      Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
  ])
  return result

actual conversion of all video files in the train directory

In [ ]:
list_of_videos = os.listdir(cmpd_train_video_path) 
videos = []
labels = []
for file_t in list_of_videos:
  t_video = cmpd_train_video_path + "/" + file_t
  cap = cv2.VideoCapture(t_video)
  frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  interval = 1
  total_duration = (frameCount + fps - 1) // fps
  indexes = [index for index in np.arange(0, fps, interval)]

  return_code = True
  images, included = [],[]
  img_preprocess = transform_image(224)

  for seconds in np.arange(0,total_duration + 1):
    if not return_code: 
      break
    seconds_base = int(seconds*fps)
    for index in indexes:
      cap.set(cv2.CAP_PROP_POS_FRAMES, seconds_base + index)
      return_code, frame = cap.read()
      if not return_code: 
        break
      frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame = img_preprocess(Image.fromarray(frame_rgb).convert("RGB"))
      videos.append(frame)
      labels.append(anno_dict[file_t])
  cap.release()

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


Shifting the axis of images from (3,224,224) to (224,224,3) shape

In [ ]:
for i in range(len(videos)):
  frames = videos[i]
  temp_frames = frames.permute(2,1,0)
  videos[i] = temp_frames


In [ ]:
videos_np = [t.numpy() for t in videos]
videos_np = np.array(videos_np)

In [ ]:
videos_np.shape

(3495, 224, 224, 3)

In [ ]:
labels_np = np.array(labels)

In [ ]:
label_bin = LabelBinarizer()
labels_np = label_bin.fit_transform(labels)

In [ ]:
train_aug = ImageDataGenerator(
    rotation_range=40,
    zoom_range=0.2,
    width_shift_range=0.3,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
val_aug = ImageDataGenerator()
mean = np.array([123.68,116.779,103.939], dtype="float32")
train_aug.mean = mean
val_aug.mean = mean

actual conversion of all video files in the test directory

In [ ]:
list_of_videos = os.listdir(cmpd_test_video_path) 
videos = []
for file_t in list_of_videos:
  t_video = cmpd_test_video_path + "/" + file_t
  cap = cv2.VideoCapture(t_video)
  frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  interval = 1
  total_duration = (frameCount + fps - 1) // fps
  indexes = [index for index in np.arange(0, fps, interval)]

  return_code = True
  images, included = [],[]
  img_preprocess = transform_image(224)

  for seconds in np.arange(0,total_duration + 1):
    if not return_code: 
      break
    seconds_base = int(seconds*fps)
    for index in indexes:
      cap.set(cv2.CAP_PROP_POS_FRAMES, seconds_base + index)
      return_code, frame = cap.read()
      if not return_code: 
        break
      frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      frame = img_preprocess(Image.fromarray(frame_rgb).convert("RGB"))
      images.append(frame)
  videos.append((images, file_t)) 
  cap.release()

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


formatting frames data

In [ ]:
for i in range(len(videos)):
  frames = videos[i][0]
  frames = frames[0:50]
  for j in range(len(frames)):
    frame = frames[j]
    frame = frame.permute(2,1,0)
    frames[j] = frame
  videos[i] = (frames, videos[i][1])

Res Net model

In [ ]:
orig_res_net_model = tf.keras.applications.ResNet50(
    weights="imagenet", 
    include_top=False, 
    input_tensor=Input(shape=(224,224,3)))
aug_model = orig_res_net_model.output
aug_model = AveragePooling2D(pool_size=(7,7))(aug_model)
aug_model = Flatten(name="flatten")(aug_model)
aug_model = Dense(1024, activation="relu")(aug_model)
aug_model = Dropout(0.5)(aug_model)
aug_model = Dense(len(label_bin.classes_), activation="softmax")(aug_model)

final_model = Model(inputs=orig_res_net_model.input, outputs=aug_model)

# marking resnet layer as untrainable
for layer in orig_res_net_model.layers:
  layer.trainable = False

In [ ]:
(trainX, validX, trainY, validY) = train_test_split(videos_np, labels_np,
	test_size=0.25, stratify=labels, random_state=42)

using stochastic gradient descent optimizer with loss as categorical crossentropy

In [ ]:
epochs = 50

optObj = SGD(lr=1e-3, momentum=0.8, decay=(1e-4 / epochs))
final_model.compile(loss="categorical_crossentropy", optimizer=optObj,
	metrics=["accuracy"])

history = final_model.fit(
	x=train_aug.flow(trainX, trainY, batch_size=32),
  validation_data=val_aug.flow(validX, validY),
	validation_steps=len(validX) // 32,
	steps_per_epoch=len(trainX) // 32,
	epochs=epochs)


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/50
81/81 [==============================] - 44s 406ms/step - loss: 1.6636 - accuracy: 0.2835 - val_loss: 1.4001 - val_accuracy: 0.4618
Epoch 2/50
81/81 [==============================] - 34s 423ms/step - loss: 1.4604 - accuracy: 0.3801 - val_loss: 1.3316 - val_accuracy: 0.4815
Epoch 3/50
81/81 [==============================] - 34s 424ms/step - loss: 1.3865 - accuracy: 0.4214 - val_loss: 1.3092 - val_accuracy: 0.4826
Epoch 4/50
81/81 [==============================] - 31s 379ms/step - loss: 1.3337 - accuracy: 0.4654 - val_loss: 1.2467 - val_accuracy: 0.5231
Epoch 5/50
81/81 [==============================] - 31s 387ms/step - loss: 1.2878 - accuracy: 0.4936 - val_loss: 1.1931 - val_accuracy: 0.5961
Epoch 6/50
81/81 [==============================] - 31s 379ms/step - loss: 1.2660 - accuracy: 0.5033 - val_loss: 1.1688 - val_accuracy: 0.5613
Epoch 7/50
81/81 [==============================] - 31s 383ms/step - loss: 1.2201 - accuracy: 0.5299 - val_loss: 1.1508 - val_accuracy: 0.6007

Saving the weights of model

In [ ]:
final_model.save(root_path + "/res_net_model_1024")

Loading weights of the model

In [7]:
final_model = tf.keras.models.load_model(root_path + "/res_net_model_1024")

extracting reduced representation vectors

In [ ]:
import keras.backend as K
get_all_layer_outputs = K.function([final_model.layers[0].input],
                                  [layer.output for layer in final_model.layers[1:]])

video_simi = []
for frames in videos:
  vectors = []
  for frame in frames[0]:
    frame = np.array([frame.numpy()])
    print(frame.shape)
    layer_output = get_all_layer_outputs([frame])
    vectors.append(layer_output[-3].flatten())
  video_simi.append((vectors,frames[1]))

In [ ]:
len(video_simi[0][0][2])

1024

storing vectors as inverted indexes for videos

In [ ]:
import csv
with open(root_path + '/inverted_index_1024.csv','w', encoding='utf-8') as out:
    csv_out=csv.writer(out, delimiter=";")
    for i in range(len(video_simi)):
      frames = video_simi[i][0]
      row_f = []
      for row in frames:
          v = ",".join(map(str, row.tolist()))
          row_f.append(v)
      row_f.append(video_simi[i][1])
      csv_out.writerow(row_f)

reading inverted indexes from file

In [7]:
import csv
csv_file_path = root_path + "/inverted_index_1024.csv"
inverted_index_read = []
with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=";")
    for row in reader:
      line = []
      for i in range(len(row) - 1):
        cell = row[i]
        vector = cell.split(",")
        vector = [float(i) for i in vector]
        line.append(vector)
      inverted_index_read.append((line,row[-1]))

In [ ]:
len(inverted_index_read)

55

In [ ]:
counter = 0
for item in inverted_index_read:
  print(counter, item[1])
  counter += 1

0 video307.mp4
1 video544.mp4
2 video171.mp4
3 video225.mp4
4 video207.mp4
5 video216.mp4
6 video222.mp4
7 video574.mp4
8 video631.mp4
9 video255.mp4
10 video194.mp4
11 video543.mp4
12 video315.mp4
13 video575.mp4
14 video739.mp4
15 video823.mp4
16 video214.mp4
17 video598.mp4
18 video173.mp4
19 video751.mp4
20 video730.mp4
21 video678.mp4
22 video645.mp4
23 video189.mp4
24 video185.mp4
25 video153.mp4
26 video660.mp4
27 video243.mp4
28 video259.mp4
29 video402.mp4
30 video619.mp4
31 video652.mp4
32 video237.mp4
33 video250.mp4
34 video230.mp4
35 video251.mp4
36 video668.mp4
37 video496.mp4
38 video474.mp4
39 video329.mp4
40 video202.mp4
41 video285.mp4
42 video149.mp4
43 video154.mp4
44 video145.mp4
45 video274.mp4
46 video539.mp4
47 video326.mp4
48 video227.mp4
49 video386.mp4
50 video500.mp4
51 video325.mp4
52 video296.mp4
53 video152.mp4
54 video318.mp4


different query images

In [14]:
query = inverted_index_read[23] # for category 9
# query = inverted_index_read[13] # for category 11
# query = inverted_index_read[25] # for category 16
# query = inverted_index_read[17] # for category 8
# query = inverted_index_read[24] # for category 0
# query

In [20]:
all_queries = [
    inverted_index_read[23],
    inverted_index_read[13],
    inverted_index_read[25],
    inverted_index_read[17],
    inverted_index_read[24]
]

In [15]:
query[-1]

'video189.mp4'

Cosine similarity function

In [10]:
def cosine_sim(a,b):
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  return cos_sim

function to fetch top k results after applying cosine similarity on dataset

In [11]:
def get_top_n(query, dataset, n_v = 10):
  results = []
  for item in dataset:
    vec_l, name = item
    key = name.split(".")[0]
    simi = 0
    for i in range(len(query[0]) - 1):
      simi += cosine_sim(vec_l[i],query[0][i])
    simi = simi / (len(query[0]) - 1)
    results.append((simi, name, anno_dict[name]))
  results = sorted(results,key=lambda x: x[0], reverse=True)
  return results

In [17]:
results = get_top_n(query, inverted_index_read)

Code used to export results for combined model

In [21]:
results_file = {}
for query in all_queries:
  query_v, file_name = query
  result = get_top_n(query, inverted_index_read, n_v=55)
  results_file[file_name] = result

with open(root_path + "/query_results_model_video_250.json", 'w') as comb:
     comb.write(json.dumps(results_file))


In [18]:
results[0:11]

[(1.0, 'video189.mp4', 9),
 (0.6703097338556789, 'video216.mp4', 0),
 (0.6626272010706061, 'video227.mp4', 0),
 (0.6347418613162646, 'video307.mp4', 16),
 (0.6286260364620005, 'video207.mp4', 0),
 (0.6188768395634379, 'video315.mp4', 16),
 (0.6157986347815437, 'video237.mp4', 0),
 (0.610284672096311, 'video251.mp4', 9),
 (0.6077471999035507, 'video243.mp4', 9),
 (0.5957739595941812, 'video202.mp4', 16),
 (0.5899047874328825, 'video259.mp4', 9)]

In [ ]:
results1[0:11]

[(1.0, 'video575.mp4', 11),
 (0.7851996240041729, 'video645.mp4', 11),
 (0.7582690577413368, 'video730.mp4', 11),
 (0.7167215237736637, 'video214.mp4', 16),
 (0.7056470414466052, 'video619.mp4', 11),
 (0.6959792146340015, 'video225.mp4', 16),
 (0.6697355814744039, 'video539.mp4', 11),
 (0.6571184797728028, 'video318.mp4', 16),
 (0.6535704670103986, 'video227.mp4', 0),
 (0.6392935688164175, 'video149.mp4', 9),
 (0.6320930020639072, 'video259.mp4', 9)]

In [ ]:
results2[0:11]

[(1.0, 'video153.mp4', 16),
 (0.6530558987045518, 'video207.mp4', 0),
 (0.6312752594004393, 'video227.mp4', 0),
 (0.6307707617222873, 'video173.mp4', 16),
 (0.6254667976357589, 'video315.mp4', 16),
 (0.6245295374842468, 'video259.mp4', 9),
 (0.6183636806777882, 'video222.mp4', 16),
 (0.6089380267570451, 'video251.mp4', 9),
 (0.6043947408815741, 'video237.mp4', 0),
 (0.5992221539736123, 'video202.mp4', 16),
 (0.5972405030375064, 'video250.mp4', 9)]

In [ ]:
results3[0:11]

[(1.0, 'video598.mp4', 8),
 (0.7815737509595672, 'video668.mp4', 8),
 (0.7670388339530188, 'video285.mp4', 16),
 (0.7423009707854442, 'video326.mp4', 0),
 (0.72441439714572, 'video539.mp4', 11),
 (0.7197196246607542, 'video296.mp4', 9),
 (0.7151423245848995, 'video543.mp4', 11),
 (0.7022370594513427, 'video318.mp4', 16),
 (0.6869392680911509, 'video474.mp4', 8),
 (0.6855807971152071, 'video386.mp4', 8),
 (0.6482417436959499, 'video152.mp4', 0)]

In [ ]:
results4[0:11]

[(1.0, 'video185.mp4', 0),
 (0.7474358867325863, 'video227.mp4', 0),
 (0.7402216872031733, 'video194.mp4', 0),
 (0.7084008066348768, 'video619.mp4', 11),
 (0.6715736667098557, 'video202.mp4', 16),
 (0.6688757029342373, 'video645.mp4', 11),
 (0.6660583969079373, 'video207.mp4', 0),
 (0.654759946956365, 'video325.mp4', 0),
 (0.6267270058823632, 'video222.mp4', 16),
 (0.6153326478482293, 'video216.mp4', 0),
 (0.6108932775763013, 'video214.mp4', 16)]

In [ ]:
annotations